In [10]:
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import (
    Agent,
    Runner,
    OpenAIChatCompletionsModel,
    ModelProvider,
    Model,
    RunConfig,
    set_default_openai_client,
    set_default_openai_api,
    set_tracing_disabled,
    function_tool,
)
load_dotenv()

True

In [11]:
client = AsyncOpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)


In [12]:
model = "gpt-4o-mini"

class GitHubModelProvider(ModelProvider):
    def get_model(self, model_name) -> Model:
        return OpenAIChatCompletionsModel(model=model_name, openai_client=client)

GITHUB_MODEL_PROVIDER = GitHubModelProvider()

In [ ]:
agent = Agent(
    name="Assistant",
    instructions="Answer the user's questions.",
)

Use the Runner class to execute the agent.


When using the run methond in the Runner, you pass in a starting _agent_ and _input_. 
The input can either be a string (which is considered a user message), or a list of input items, which are the items in the OpenAI Responses API.

The runner then runs a loop:

1. call the LLM for the current agent, with the current input.
2. The LLM produces its output.
   1.  If the **LLM returns a final_output**, the loop ends and we return the result.
   2.  If the LLM does a handoff, we update the current agent and input, and re-run the loop.
   3.  If the LLM produces tool calls, we run those tool calls, append the results, and re-run the loop.
3. If we exceed the max_turns passed, we raise a MaxTurnsExceeded exception.


The rule for whether the LLM output is considered as a "final output" is that it produces text output with the desired type, and there are no tool calls.

In [9]:
# https://github.com/openai/openai-agents-python/blob/main/examples/model_providers/custom_example_provider.py

result = await Runner.run(
    agent,
    "Tell me a joke?",
    run_config=RunConfig(model_provider=GITHUB_MODEL_PROVIDER, model=model),
)

print(result.final_output)

Why did the scarecrow win an award? 

Because he was outstanding in his field!
